<a href="https://colab.research.google.com/github/phanisrirajyalakshmikeerthi/NEURAL-NETWORK-TRANSFER/blob/main/MY_PROJECT_DLWTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import random
import numpy as np

# Seed for reproducibility
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Define a transform to preprocess the images
transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(112),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load a small subset of the CIFAR-10 dataset (reduce training time)
train_dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=transform)
subset_indices = torch.randperm(len(train_dataset))[:1000]  # Only use 1000 samples
train_subset = torch.utils.data.Subset(train_dataset, subset_indices)
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=64, shuffle=True)

# Load a pre-trained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)

# Freeze the parameters
for param in model.parameters():
    param.requires_grad = False

# Modify the final layer
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 10)  # CIFAR-10 has 10 classes

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Train the model
num_epochs = 2  # Reduce epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

print("Training complete!")


Files already downloaded and verified
Epoch [1/2], Loss: 1.9906
Epoch [2/2], Loss: 1.2421
Training complete!


In [3]:
# Load the CIFAR-10 test dataset
test_dataset = datasets.CIFAR10(root='data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

# Evaluate the model
model.eval()  # Set the model to evaluation mode

correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the CIFAR-10 test images: {100 * correct / total:.2f}%')


Files already downloaded and verified
Accuracy of the network on the CIFAR-10 test images: 65.16%
